In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = r"C:\Users\dewald.brand\AVIS\Revenue Forecast - General\Replacement Calculation\Monthly Datav1.xlsx"

In [3]:
# Use a context manager to open and read the Excel file
with pd.ExcelFile(file_path) as excel_data:
    # List all sheet names
    sheet_names = excel_data.sheet_names    

    # Load each sheet into a DataFrame
    data_frames = {sheet: excel_data.parse(sheet) for sheet in sheet_names}


In [4]:
for sheet_name, df in data_frames.items():
    df['SheetName'] = sheet_name
    

In [5]:
df = pd.concat(data_frames.values(), ignore_index=True)

In [6]:
df['MVA NUMBER'] = df['MVA NUMBER'].astype('int')

In [7]:
sales = df.loc[(df['COMPANY CODE'] == 'RSA')]

In [8]:
sales = sales.sort_values(by='SALES EXECUTIVE NAME', na_position='last')

In [9]:
sales = sales.loc[sales['SALES EXECUTIVE NAME'].notna()]

In [10]:
sales = sales[['CUSTOMER TRADING NAME', 'SALES EXECUTIVE NAME']]

In [11]:
sales = df.loc[(df['COMPANY CODE'] == 'RSA') & (df['SALES EXECUTIVE NAME'].notna())]

In [12]:
sales = sales.drop_duplicates()

In [23]:
# Step 1: Filter for RSA
rsa_sales = sales[sales['COMPANY CODE'] == 'RSA']

# Step 2: Identify repeat customers
customer_counts = rsa_sales.groupby(['CUSTOMER TRADING NAME', 'MVA NUMBER']).size()
repeat_customers = customer_counts[customer_counts > 1].reset_index()[['CUSTOMER TRADING NAME', 'MVA NUMBER']]

# Step 3: Filter repeat customer sales
repeat_sales = pd.merge(rsa_sales, repeat_customers, on=['CUSTOMER TRADING NAME', 'MVA NUMBER'])

# Step 4: Calculate total and repeat rental revenue
total_rental_revenue = rsa_sales['RENTAL AMOUNT'].sum()
repeat_rental_revenue = repeat_sales['RENTAL AMOUNT'].sum()

# Step 5: Calculate percentage
repeat_percentage = (repeat_rental_revenue / total_rental_revenue) * 100 if total_rental_revenue > 0 else 0

# Output result
print(f"Percentage of RSA rental revenue attributed to repeat business: {repeat_percentage:.2f}%")


Percentage of RSA rental revenue attributed to repeat business: 89.71%
